In [1]:
import os
from functools import partial

import tensorflow as tf

2024-07-19 17:46:48.150724: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
def func():
    return 1, 2, 3

In [7]:
a, b = tf.constant([[1, 2], [3, 4]])

In [8]:
a, b

(<tf.Tensor: shape=(2,), dtype=int32, numpy=array([1, 2], dtype=int32)>,
 <tf.Tensor: shape=(2,), dtype=int32, numpy=array([3, 4], dtype=int32)>)

In [6]:
a = tf.constant([[1, 2]])
print(a.shape)

(1, 2)


In [4]:
tf.reshape(a, shape=(2,))

<tf.Tensor: shape=(2,), dtype=int32, numpy=array([1, 2], dtype=int32)>

In [ ]:
def _parse(proto, ft_desc):
    parsed_ft = tf.io.parse_single_example(proto, ft_desc)
    coord = parsed_ft["coord"]
    label = parsed_ft["label"]
    oh_label = parsed_ft["oh_label"]
    image = parsed_ft["logit"]

    return image, coord, label, oh_label

def load_data_tfrecord(
    path_to_tfrecord="/media/daniel/DatasetIMDB/imdb_chunks", class_num: int = 8
):
    tfrecord_files = [
        os.path.join(path_to_tfrecord, f)
        for f in os.listdir(path_to_tfrecord)
        if f.endswith(".tfrecord")
    ]

    feature_description = {
        "logit": tf.io.FixedLenFeature([], tf.string),
        "label": tf.io.FixedLenFeature([], tf.float32),
        "oh_label": tf.io.FixedLenFeature([class_num], tf.float32),
        "coord": tf.io.FixedLenFeature([4], tf.float32),  # min_x, max_x, min_y, max_y
    }
    parse_fn = partial(_parse, ft_desc=feature_description)

    raw_dataset = tf.data.TFRecordDataset(tfrecord_files)
    parsed_dataset = raw_dataset.map(parse_fn)

    return parsed_dataset

In [ ]:
for batch in load_data_tfrecord():
    logits, coord, *_ = batch

    print(tf.reshape(
        tf.cast(tf.io.decode_raw(logits, tf.uint8), tf.float32),
        shape=(-1,) + (224, 224, 3),
    ))

    print(coord)
    break